In [1]:
import numpy as np
import matplotlib.pyplot as plt
import mne

In [2]:
%matplotlib inline

# Frequency-Filtering Signals

**Exercises**

#### Create the Data

Using Numpy and Matplotlib, plot 3 seconds-worth of three sine waves: a 3 Hz wave, an 8 Hz wave, and a 20 Hz wave.  Use the formula below ($f$ is the frequency in Hz, and $t$ is the time in seconds)

$$  \sin{f 2 \pi t}  $$

Adjust the plot so that you can see all of the waves clearly and each signal.  If using a legend, this will let you move it out of the way: `plt.legend(loc='upper left', bbox_to_anchor=(1, 1));`

Make and plot another signal, that's the mean of the three sine waves.

Plot the signals with MNE: Make an mne.Raw data structure from these four signals (the individual waves and the combined wave).  Reminder: The functions `mne.io.RawArray()` and `mne.create_info()` will be helpful here.  

### Frequency Band Filters: Lowpass, Highpass, Bandpass, and Bandstop

The `Raw.filter()` function filters all the picked channels in the dataset.  It has a lot of parameters for customizing the filter, but we'll just focus on two: `l_freq` (lowest frequency allowed) and `h_freq` (highest frequency allowed):

  - **Highpass Filter**: Keep everything above a certain frequency. `Raw.filter(l_freq=2, h_freq=None`)
  - **Lowpass Filter**: Keep everything above a certain frequency.  `Raw.filter(l_freq=None, h_freq=20`)
  - **Bandpass Filter**: Keep everything between two frequencies.  `Raw.filter(l_freq=2, h_freq=20`)
  - **Bandstop Filter**: Reject everything between two frequencies.  `Raw.filter(l_freq=20, h_freq=2`)
  
In the exercises below, you'll make different filters and apply them to the data.  Besides learning how MNE works, the goal here is to get a feel for the imperfections of filters, so experiment and keep plotting the data!

*Tip*: `Raw.filter()` modifies the data in-place.  To avoid re-filtering the same data twice, the following snippet can help: `raw.copy().filter()`

*Tip*: If the channels are **"Misc"**, you can pick them to be filtered with `Raw.filter(picks='misc')`

*Tip*: Filtering can re-scale the data, which can make plotting a challenge.  Set a vertical compression factor when plotting with `Raw.plot(scalings=2)`

**Exercises**: Plot filtered signals with the requested properties

Filter out the 3-Hz wave, keep the other signals intact

Filter out the 20-Hz wave, keep the other signals intact

Filter out the 3-Hz and 8-Hz wave, keep the other signals intact

Filter out the 8-Hz wave, keep the other signals intact

### Notch Filter

`Raw.notch_filter([50])` It's also possible to specify exact frequencies to filter out; this is called a "notch filter", and is often used to filter out powerline frequencies (50 Hz in Europe, 60 Hz in U.S.). 

**Exercises**: Use the notch filter to filter out the requested frequencies

Filter out the 20-Hz wave, keep the other signals intact

Filter out the 8-Hz wave, keep the other signals intact

Filter out the 3-Hz and 8-Hz wave, keep the other signals intact

### Examining Filter Properties

What does this filter actually look like, and how is it expected to impact the power of different frequencies, and timing of the signals?

MNE's function `mne.filter.create_filter()` returns the Numpy array with the filter shape, customized for the properties of your data.  This can then be examined either by plotting it with `plt.plot()`, or even better, with `mne.viz.plot_filter()`

*Tip*: `mne.filter.create_filter()` needs the numpy array and sfreq extracted from the Raw object.  

**Exercises**

Using Matplotlib, plot a 2-Hz Lowpass filter created for the Raw data.

Plot the filter properties with MNE.  Does this filter affect all frequencies the same?

Examine a 5-20 Hz Bandpass filter created for the Raw data.

Recreate the Raw data with 60-seconds of signal, and re-plot the 2-Hz lowpass filter.  How is it changed?

### Resampling Best Practices in MNE

Data is often recorded at a higher sampling rate than what is required for the final analysis. To reduce computational requirements, data can be resampled to a lower sampling rate.

Resampling can be performed using the built-in `.resample()` method of the `Raw`, `Epochs`, and `Evoked` objects, or the standalone `mne.filter.resample()` function.

**Exercises**

Create a copy of the `Raw` data and resample it to 100 Hz using the built-in `Raw.resample()` method.

In [ ]:
raw_copy = raw.copy()

# Add code here...

Plot the power spectra for the resampled data. What is the new upper limit of the spectrum and why?

Inspect the start and end of the data. Do you notice anything strange?

Now create a set of 2 second-long epochs from the original `Raw` object and resample it to 100 Hz using the built-in `Epochs.resample()` method.

*Hint*: `mne.make_fixed_length_epochs()` will be useful.

Now inspect the epochs. How does this compare to the `Raw` data?

What you see are edge artefacts of the filtering introduced when resampling.

As you can see, resampling `Epochs` data can introduce edge artefacts in **every** epoch! In contrast, resampling `Raw` data only introduces an artefact at the start and end of the recordings.

However, resampling `Raw` objects can reduce the temporal precision of associated event arrays, which is not an issue if epoching first.

To combat both these problems, it is recommended that you:
1. Low-pass filter `Raw` data at <= 1/3 the desired sampling rate, then...
2. Decimate the `Epochs` data using the `decim` parameter when constructing the object, or using the `.decimate()` method after the object has been created.

Now use this recommended procedure to generate the 100 Hz-resampled epochs. How do the epochs compare?

If it is not possible to follow the above procedure, you should at least follow these recommendations:

- If you have to resample `Raw` data, pass in any corresponding event markers using the `events` parameter when resampling to mitigate a loss of synchrony between the data and events, e.g. `new_raw, new_events = raw.resample(sfreq, events=events)`.

or

- If you have to resample `Epochs` data, make the epoch durations long enough such that the edge artefacts in each epoch will not affect the data you wish to analyse.

## Further Reading

MNE has an excellent article on the theory of filtering.
  - Link Here: https://mne.tools/stable/auto_tutorials/preprocessing/25_background_filtering.html
  - [Download it as a Jupyter notebook](https://mne.tools/stable/_downloads/6d98b103d247000f4433763dd76607c0/25_background_filtering.ipynb)

An overview of filtering and resampling is also presented here: https://mne.tools/stable/auto_tutorials/preprocessing/30_filtering_resampling.html